In [4]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic import hub

load_dotenv()

True

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [37]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [39]:
loader = PyPDFLoader("important Placement Questions.pdf", extract_images=True)
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=25, chunk_overlap=15)
splits = text_splitter.split_documents(docs)

In [40]:
print(f"Number of pages loaded: {len(docs)}")
if len(docs) > 0:
    print(f"Sample content from page 1: {docs[0].page_content[:100]}")
else:
    print("!!! ERROR: No text found in PDF. Check if the file path is correct.")
    

Number of pages loaded: 10
Sample content from page 1: grow_pins
EssentialInterviewQuestionsforPython,Java,DataStructures,DBMS,andWebTechnologies.
Masterth


In [41]:
print(f"Number of chunks created: {len(splits)}")
if len(splits) == 0:
    print("!!! ERROR: Text splitter produced 0 chunks. Try reducing chunk_size.")

Number of chunks created: 912


In [24]:
if len(splits) > 0:
    vectorstore = Chroma.from_documents(
        documents=splits, 
        embedding=embeddings,
        persist_directory="./chroma_db"
    )
    print("Successfully created vector store.")
else:
    raise ValueError("The splits list is empty! Ensure your PDF loader actually found text.")

Successfully created vector store.


In [42]:
prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(vectorstore.as_retriever(), combine_docs_chain)

In [44]:
response = rag_chain.invoke({"input": "What are the main points of this document?"})
response = rag_chain.invoke({"input": "What are the main points of this document in python?"})

print(response["answer"])

The main points of this document related to Python are:

*   **Python's Core Features:** It is interpreted, dynamically typed, object-oriented, supports multiple paradigms, and has an extensive standard library.
*   **Data Structures:** Differences between lists (mutable, more memory) and tuples (immutable), and between shallow and deep copies.
*   **Memory Management:** Python uses reference counting and garbage collection.
*   **Resource Management:** The `with` statement ensures proper resource acquisition and release, particularly for file handling (`open()`, `read()`, `write()`, `close()`).
*   **Control Flow:** Loop control statements include `break`, `continue`, and `pass`.
*   **Concurrency:** Explanation of the Global Interpreter Lock (GIL).
*   **Functions:** Lambda functions (anonymous functions), and a mention of `map()`, `filter()`, and `reduce()`.
*   **List Comprehension:** How to generate lists concisely (e.g., `[x**2 for x in range(5)]`).
*   **Code Organization:** Def

In [45]:

response = rag_chain.invoke({"input": "What are the main points of this document in java?"})

print(response["answer"])

The main points of this document related to Java are:

*   **Fundamentals:** Definition of Java, purpose of the `main()` method.
*   **Java Environment:** Differences between JDK, JRE, and JVM.
*   **Object-Oriented Programming (OOP) Concepts:** Inheritance, polymorphism (compile-time and runtime), and abstraction (abstract classes and interfaces).
*   **Memory Management:** Garbage collection.
*   **Keywords and Operators:** Differences between `this` and `super`, and `==` vs `.equals()`.
*   **Concurrency:** How multithreading works.
*   **Access Control:** Java's access modifiers (`private`, `default`, `protected`, `public`).
*   **String Manipulation:** Differences between `StringBuilder` and `StringBuffer`.
*   **Interfaces vs. Abstract Classes:** Their key differences.
